In [1]:
%matplotlib inline

import cPickle as pickle
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
import gensim as gs
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
import lda
from load import all_subreddits_data, tv_subreddits_data


import matplotlib.pyplot as plt



Bad key "axes.spines.right" on line 210 in
//anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://matplotlib.sf.net/_static/matplotlibrc or from the matplotlib source
distribution

Bad key "boxplot.flierprops.linewidth" on line 321 in
//anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://matplotlib.sf.net/_static/matplotlibrc or from the matplotlib source
distribution

Bad key "contour.corner_mask" on line 306 in
//anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://matplotlib.sf.net/_static/matplotlibrc or from the matplotlib source
distribution

Bad key "boxplot.vertical" on line 339 in
//anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get

In [2]:
# data is of the form {class_label: list of documents}
all_data = all_subreddits_data(30, 500, 10)
# tv_data = tv_subreddits_data()

KeyboardInterrupt: 

In [3]:
map(len, all_data.values())
# all_data.keys()
# all_data2 = {'buildapc': all_data['buildapc'], 'anime': all_data['anime']}

[500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 486,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500]

In [4]:
def tt_split(data, test_size=.1):
    """Splits a dictionary {class_label: list of documents}"""
    """into two dictionaries of the same shape"""
    train_data = {}; test_data = {}
    for label, docs in data.iteritems():
        train, test = train_test_split(docs, test_size=test_size)
        train_data[label] = train
        test_data[label] = test
    return train_data, test_data
        
        

In [5]:
def tokens_to_vocab(class_tokens):
    """{class_label : list of tokenized documents} -> vocab"""
    vocab = set([])
    for _class, tokenized_docs in class_tokens.iteritems():
        for d in tokenized_docs:
            vocab = vocab.union(set(d))
    return {word: i for i, word in enumerate(vocab)}
        

def word_tokenize_doc(doc):
    """Word tokenize a single document"""
    to_remove = set(['http', 'faq', 'https', 'amp','source', 'deletion', 'sfw',
              'nsfw', 'gt', 'gon', 'na', 'delete', 'comment', 'profile'])
    def _filter(w):
        return all([w.isalnum(), w not in stopwords.words('english'), w not in to_remove])
    tokens = word_tokenize(doc)
    tokens = filter(_filter, tokens)
    return tokens

def tokenize_all_words(data):
    """basic get_tokens method"""
    """{class_label: list of documents} ->""" 
    """{class_label : list of tokenized documents}"""
    for c, docs in data.iteritems():
        data[c] = map(word_tokenize_doc, docs)
    return data

# test_data = {
#     'c1': ['aa bb c', 'b d'],
#     'c2': ['d e f', 'e g']
# }
# test_tokens = tokenize_all_words(test_data)
# test_vocab = tokens_to_vocab(test_tokens)


In [6]:
# def get_lda_topics(train_tokens, vocab, n_topics=30):
#     """{class_label: list of tokenized docs} ->"""
#     """{class_label: list of topic distributions from LDA"""
#     all_class_tokens = train_tokens.values()
#     flattened_classes = [item for sublist in all_class_tokens for item in sublist]
#     flattened_documents = [item for sublist in flattened_classes for item in sublist]
#     vectorizer = CountVectorizer(min_df=2, vocabulary=vocab)
#     X = vectorizer.fit_transform(flattened_documents)
#     lda = LatentDirichletAllocation(n_topics=n_topics)
#     X_new = lda.fit_transform(X.toarray())
#     return X_new

def get_hlda_models(train_tokens, vocab, n_topics=40):
    models = {}
    dictionary = gs.corpora.Dictionary(map(lambda x: [x], vocab.keys()))
    for label, docs in train_tokens.items():
        corpus = [dictionary.doc2bow(d) for d in docs]
        models[label] = gs.models.HdpModel(corpus, dictionary, T=n_topics)
    return models, dictionary

def get_lda_models(train_tokens, vocab, n_topics=40):
    all_models = {}
    def fit_model((label, docs)):
        model = lda.LDA(n_topics=n_topics, n_iter=200)
        vectorizer = CountVectorizer(min_df=2, vocabulary = vocab, stop_words=None)
        X = vectorizer.fit_transform(map(lambda s: ' '.join(s), docs))
        model.fit(X)
        all_models[label] = model
        print 'done fitting for ', label
    map(fit_model, train_tokens.items())
    return all_models


def hlda_pred(models, dictionary, doc):
    corpus = [dictionary.doc2bow(word_tokenize_doc(doc))]
    label_score = []
    for label, hdp in models.iteritems():
        label_score.append((label, hdp.evaluate_test_corpus(corpus)))
    return max(label_score, key = lambda x:x[1])[0]

def lda_pred(models, vocab, doc):
    """Get a class prediction for a document """
    tokenized = word_tokenize_doc(doc)
    vectorizer = CountVectorizer(min_df=1, vocabulary = vocab, stop_words=None)
    X = vectorizer.fit_transform([' '.join(tokenized)])
    label_score = []
    for label, model in models.iteritems():
        n_topics = len(model.components_)
        topic_dist = model.transform(X)
        log_likelihood = 0
        for token in tokenized:
            if token in vocab:
                max_likelihood = -1 * 10 ** 8
                for topic in range(n_topics):
                    ll = np.log(model.components_[topic][vocab[token]]) + np.log(topic_dist[0][topic])
                    max_likelihood = max_likelihood if max_likelihood > ll else ll
                log_likelihood += max_likelihood
        label_score.append((label, log_likelihood))
    return max(label_score, key = lambda x:x[1])[0]
# get_lda_topics(test_tokens, test_vocab)

In [7]:
# def run_model(data, get_tokens=tokenize_all_words,
#               get_models = get_lda_models):
#     """Vectorizes, topic models, classifies, and returns score"""
data = all_data
get_tokens = tokenize_all_words
get_models = get_lda_models

train, test = tt_split(data)
print 'done splitting'
train_tokens = get_tokens(train)
print 'done tokenizing'
vocab = tokens_to_vocab(train_tokens)



done splitting
done tokenizing


In [2]:
# lda_models = get_lda_models(train_tokens, vocab)

# dump 
pickle_filepath = 'cache/data.pickle'

def dump():
    with open(pickle_filepath, 'w') as wfile:
        pickle.dump( (train, test, train_tokens, vocab), wfile)

def load():
    with open(pickle_filepath, 'r') as rfile:
        train, test, train_tokens, vocab = pickle.load(rfile)
    return train, test, train_tokens, vocab
    

In [3]:
train, test, train_tokens, vocab = load()

In [27]:
# correct = 0
# total = 0
# for label, docset in test.iteritems():
#     for doc in docset:
#         total += 1
#         if lda_pred(lda_models, vocab, doc) == label:
#             correct += 1
# print correct, total
from collections import defaultdict
lookup = {}
words = set()
for label, docs in train_tokens.iteritems():
    counts = defaultdict(int)
    for d in docs:
        for t in d:
            counts[t] += 1
            words.add(t)
    lookup[label] = counts


In [9]:
from collections import defaultdict
from scipy.stats import dirichlet
def normalize(v):
    return v / float(np.sum(v))

def score_words(train_tokens):

    words = set()
    all_counts = {}
    for label, docs in train_tokens.iteritems():
        counts = defaultdict(int)
        for d in docs:
            for t in d:
                counts[t] += 1
                words.add(t)
        all_counts[label] = counts


    # d filter
    word_score = []
    for w in words:
        get_count = lambda d: d.get(w, 0)

        x = normalize(np.array(map(get_count, all_counts.values())))
        score = dirichlet.logpdf(normalize(x), np.ones(len(all_counts)) * 2)
        word_score.append((w, score))
    return word_score


def dirichlet_filter(train_tokens, threshold = 20):

    word_scores = score_words(train_tokens)

    words_to_remove = set(map(lambda x:x[0], filter(lambda x: x[1] > threshold, ws)))

    keep_word = lambda w: w not in words_to_remove

    removed = 0
    total = 0

    for label, docs in train_tokens.iteritems():
        for i in range(len(docs)):
            old_length = len(docs[i])
            total += old_length
            docs[i] = filter(keep_word, docs[i])
            removed += old_length - len(docs[i])
        train_tokens[label] = docs

    return train_tokens, removed, total

In [10]:
# train_tokens, removed, total = dirichlet_filter(train_tokens)
word_score = score_words(train_tokens)

//anaconda/lib/python2.7/site-packages/scipy/stats/_multivariate.py:805: RuntimeWarning: divide by zero encountered in log
  return - lnB + np.sum((np.log(x.T) * (alpha - 1)).T, 0)


In [20]:
from string import lower
for w in set(map(lower, map(lambda x: x[0], sorted(word_score, key=lambda x: x[1], reverse=True)[:200]))):
    print w

forget
just
yea
four
ended
stayed
go
hell
still
hate
had
cuz
better
only
going
pretty
sorry
might
ahh
surprised
ta
good
holding
yo
very
looks
break
worst
half
not
one
minute
freaking
like
lost
edit
always
twice
bad
either
went
last
blew
everyone
eh
absolutely
yeah
second
picked
forgot
close
little
surprise
really
missed
god
wow
wouldnt
seriously
goes
hilarious
got
ass
kicked
never
meh
let
although
put
post
both
ol
ok
thread
oh
dunno
leaves
barely
swear
plus
idk
threw
dis
makes
thats
dont
ca
two
downvote
next
wan
guess
yep
dude
three
lol
kidding
damn
too
6
happy
head
that
huh
took
back
catch
probably
glad
13
look
gotten
straight
getting
sucks
see
yup
gave
didnt
bet
gone
crazy
give
ai
ah
turned
confused
sad
shame
cant
someone
im
something
ready
id
shit
thought
nonetheless
nah
same
ya
amazing
sick
9
gets
nice
badly
sure
okay
hey
though
haha
stay
fell
mad
kinda
coming
awful
suck
totally
man
a
maybe
no
well
face
looked
definitely
honestly
alright
five
away


In [160]:
# print sorted(ws, key=lambda x: x[1], reverse=True)[:100]

# plt.hist(map(lambda x:x[1], ws))
# plt.show()
words = set()
for label, docs in train_tokens.iteritems():
    counts = defaultdict(int)
    for d in docs:
        for t in d:
            words.add(t)
print len(words)

243112


In [143]:
hlda_models, dictionary = get_hlda_models(train_tokens, vocab, 40)
correct = 0
total = 0
for label, docset in test.iteritems():
    for doc in docset:
        total += 1
        if hlda_pred(hlda_models, dictionary, doc) == label:
            correct += 1
print correct, total

943 1499


In [ ]:
# map(lambda (l, t): map(lambda x: x[0], t), hlda_models['aww'].show_topics(formatted=False))

In [ ]:
# print pred(Out[30], vocab, 'qi')
# print pred(Out[30], vocab, 'gold')
hlda_models.keys()

In [ ]:
lda_models = get_lda_models(train_tokens, vocab)

In [ ]:
correct = 0
total = 0
for label, docset in test.iteritems():
    for doc in docset:
        total += 1
        if lda_pred(lda_models, vocab, doc) == label:
            correct += 1
print correct, total

In [ ]:
correct, total